# QLMC : Q-Learning-Based Model Checking
Case Study: Dining Philosophers model with N Philsophers and N Chopsticks

In [27]:
"""
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2
"""

'\n!pip install tensorflow==2.3.0\n!pip install gym\n!pip install keras\n!pip install keras-rl2\n'

In [13]:
from gym import Env
from gym import spaces
#from gym.spaces import Discrete, Box
import numpy as np
import numpy.ma as ma
import random
import tqdm
from tqdm.notebook import tqdm
import copy
#import maude

In [89]:
class DiningPhilEnv(Env):
    def __init__(self, N, vec=None):
        self.N = N # number of philosophers = number of chopsticks
        #self.P = np.zeros(shape=(N,), dtype=int)
        #self.C = np.zeros(shape=(N,), dtype=int)
        if vec != None:
            # TODO assert vec has len 2N
            self.last_action = -1
            self.P = np.array(vec[:N])
            self.C = np.array(vec[N:])
            self.next_actions = self._get_next_actions()
        else:
            self.reset()
        
        self.observation_space = spaces.Dict(
            {
                "P": spaces.Box(0, 3, shape=(N,), dtype=int),
                "C": spaces.Box(0, 1, shape=(N,), dtype=int),
            }
        )
        self.action_space = spaces.Discrete(4*N)
        #self.abs_action_space = spaces.Box(low=0, high=1, shape=(2, 2, 2, 2), dtype=int)
        self.shape_state_space = ((4,)* self.N) + ((2,)* self.N)
        self.shape_action_space = (4 * self.N,)
        #self._generate_random_state()
        
    def step(self, a):
        # TODO: refactoring: replace below with _step
        self.last_action = a
        phil_id = a // 4
        action_type = a % 4
        
        if action_type == 0: # think -> hungry
            self.P[phil_id] = 1
        elif action_type == 1: # pick left
            self.P[phil_id] += 1
            self.C[phil_id] = 0
        elif action_type == 2: # pick right
            self.P[phil_id] += 1
            self.C[(phil_id + 1) % self.N] = 0
        else: # eat -> think
            self.P[phil_id] = 0
            self.C[phil_id] = 1
            self.C[(phil_id + 1) % self.N] = 1
            
        self.next_actions = self._get_next_actions()
        observation = self._get_obs()
        reward = 0
        done = False
        if not np.any(self._get_mask()): # TODO: replace self.is_goal
            # deadlock
            reward = 1
            done = True
        info = self._get_info()
        return observation, reward, done, info
    
    def _step(self, a):
        # to avoid recursion. can be thought of as incomplete self.step()
        self.last_action = a
        phil_id = a // 4
        action_type = a % 4
        
        if action_type == 0: # think -> hungry
            self.P[phil_id] = 1
        elif action_type == 1: # pick left
            self.P[phil_id] += 1
            self.C[phil_id] = 0
        elif action_type == 2: # pick right
            self.P[phil_id] += 1
            self.C[(phil_id + 1) % self.N] = 0
        else: # eat -> think
            self.P[phil_id] = 0
            self.C[phil_id] = 1
            self.C[(phil_id + 1) % self.N] = 1
        
        vec = self._get_vec()
        abs_vec = self._get_abs_vec()
        return vec, abs_vec
    
    def is_goal(self):
        return not np.any(self._get_mask()) # deadlock
    
    def _get_obs(self):
        return {
            "vec": self._get_vec(), #tuple(self.P) + tuple(self.C),
            "abs_vec": self._get_abs_vec(),
            "mask": self._get_mask(),
            "abs_mask": self._get_abs_mask(),
        }
    
    def _get_info(self):
        return {
            #"mask": self._get_mask()
            #"mask": self._get_mask().reshape((self.N, 4))
            "action_desc" : self._get_action_desc()
        }
    
    def reset(self):
        # TODO randomize init step by chopstick pocession
        self.last_action = -1
        #self.P = np.zeros(shape=(self.N,), dtype=int)
        #self.C = np.ones(shape=(self.N,), dtype=int)
        self._generate_random_state()
        self.next_actions = self._get_next_actions()
        observation = self._get_obs()
        info = self._get_info()
        return observation, info
    
    def _get_mask(self):
        # 0: disabled, 1: enabled
        N = self.N
        P = self.P
        C = self.C
        mask = np.zeros(4 * N, dtype=np.int8)
        for phil_id in range(N):
            if P[phil_id] == 0:
                mask[4 * phil_id + 0] = 1
            elif P[phil_id] == 3:
                mask[4 * phil_id + 3] = 1
            else:
                mask[4 * phil_id + 1] = C[phil_id]
                mask[4 * phil_id + 2] = C[(phil_id + 1) % self.N]
        return mask
    
    def _get_abs_mask(self):
        # 0: disabled, 1: enabled
        abs_mask = np.zeros(shape=(2,2,2,2), dtype=np.int8)
        next_actions = self._get_next_actions()
        for _, _, abs_a in next_actions:
            abs_mask[abs_a] = 1
        return abs_mask
    
    def abs_step(self, abs_action):
        # from abstract action, step with a randomly chosen corresponding concrete action
        # assume abs_action is available i.e. there exists at least one concrete action
        actions = [a for (a, _, aa) in self.next_actions if aa == abs_action]
        action = random.choice(actions)
        return self.step(action)
    
    def _get_next_actions(self):
        # TODO: refactoring pythonic
        mask = self._get_mask()
        next_actions = []
        for action, flag in enumerate(mask):
            if flag == 1:
                vec, abs_vec = copy.deepcopy(self)._step(action) # garbage collection?
                next_actions.append((action, vec, abs_vec))
        return next_actions
                
        
    def _get_action_desc(self):
        # returns the description of the last action
        a = self.last_action
        if a == -1:
            return "no action"
        phil_id = a // 4
        action_type = a % 4
        desc = "philsopher " + str(phil_id)
        if action_type == 0: # think -> hungry
            desc += " gets hungry"
        elif action_type == 1: # pick left
            desc += " picks left chopstick"
        elif action_type == 2: # pick right
            desc += " picks right chopstick"
        else: # eat -> think
            desc += " stops eating"
        return desc
    
    def _generate_random_state(self):
        N = self.N
        self.P = np.zeros(N, dtype=int)
        self.C = np.ones(N, dtype=int)
        
        for i in range(N):
            c = random.randrange(3)
            if c == 1:
                # to left
                self.P[(i-1) % N] += 1
                self.C[i] = 0
            elif c == 2:
                # to right:
                self.P[i] += 1
                self.C[i] = 0
                
        # here, self.P[i] denotes the number of chopstics assigned for ith philos
        for i in range(N):
            if self.P[i] == 0:
                self.P[i] = random.randrange(2) # either think or hungry
            else:
                self.P[i] += 1 # one chopstick or eat

    def _get_vec(self):
        return tuple(self.P) + tuple(self.C)
                
    def _get_abs_vec(self):
        # abs_vec : < Think, Hungry, Single, Eat > e.g. < 1, 1, 1, 0 >
        return tuple(int(i in self.P) for i in range(4))
    

    

In [129]:
# BFS with heuristics given by abstract Q table

def bfs(N=5, vec=None, max_step=10000, Qtable=None):
    if not Qtable is None:
        heuristics = True
    else:
        heuristics = False
    env = DiningPhilEnv(N,vec)
    vec = env._get_vec()
    #abs_vec = env._get_abs_vec()
    #print('initial config:', vec)
    visited = set()

    i = 0
    queue = [(i,vec)] # (priority, vec)
    while not queue == []:
        vec = heapq.heappop(queue)[1]
        if vec in visited:
            continue
        i += 1
        visited.add(vec)
        env = DiningPhilEnv(N, vec)
        abs_vec = env._get_abs_vec()
        if env.is_goal():
            #print('goal reached!')
            #print('vec:', vec)
            #print('num steps:', i)
            break
        elif i == max_step:
            #print('max step reached!')
            break
        nbrs = [(v, av) for (a, v, av) in env.next_actions if not v in visited] # unvisited next vecs
        if heuristics:
            p_nbrs = [(-Qtable[abs_vec + av], v) for (v, av) in nbrs] # prioritized nbrs
        else:
            p_nbrs = [(i, v) for (v, av) in nbrs] # prioritized nbrs
        #print(p_nbrs)
        for item in p_nbrs:
            heapq.heappush(queue, item)
    return i

In [106]:
freshQtable = init_q_table((2,2,2,2), (2,2,2,2))
print('freshQtable density:', np.count_nonzero(freshQtable), '/', freshQtable.size)
print('absQtable density:', np.count_nonzero(absQtable), '/', absQtable.size)

freshQtable density: 0 / 256
absQtable density: 44 / 256


In [ ]:
def evaluateBFS(num_instances, N, max_steps, Qtable):
    res_fifo = []
    res_qlmc = []
    for i in range(num_instances):
        env = DiningPhilEnv(N)
        vec = env._get_vec()
        #print('----------')
        #print('initial config:', vec)
        num_states_fifo = bfs(N, vec, max_steps)
        res_fifo.append(num_states_fifo)
        num_states_qlmc = bfs(N, vec, max_steps, Qtable)
        res_qlmc.append(num_states_qlmc)
        #print('fifo:', num_states_fifo, ', qlmc:', num_states_qlmc)
    avg_fifo = np.average(np.array(res_fifo))
    avg_qlmc = np.average(np.array(res_qlmc))
    print('=== STAT ===')
    print('N:', N)
    print('num_instances:', num_instances)
    print('max_steps:', max_steps)
    print('avg_fifo:', avg_fifo)
    print('avg_qlmc:', avg_qlmc)


evaluateBFS(30, 5, 10000, absQtable)
evaluateBFS(30, 6, 10000, absQtable)
evaluateBFS(30, 7, 10000, absQtable)
evaluateBFS(30, 8, 10000, absQtable)
evaluateBFS(30, 9, 10000, absQtable)
evaluateBFS(30, 10, 10000, absQtable)

=== STAT ===
N: 5
num_instances: 30
max_steps: 10000
avg_fifo: 328.53333333333336
avg_qlmc: 27.3
=== STAT ===
N: 6
num_instances: 30
max_steps: 10000
avg_fifo: 1370.3
avg_qlmc: 70.93333333333334
=== STAT ===
N: 7
num_instances: 30
max_steps: 10000
avg_fifo: 4897.033333333334
avg_qlmc: 220.5
=== STAT ===
N: 8
num_instances: 30
max_steps: 10000
avg_fifo: 8173.266666666666
avg_qlmc: 725.7333333333333
=== STAT ===
N: 9
num_instances: 30
max_steps: 10000
avg_fifo: 8520.266666666666
avg_qlmc: 2299.2


In [64]:
#a = np.array([1,0,2,0])
#tuple(int(i in a) for i in range(4))
abs_mask = np.zeros(shape=(2,2,2,2), dtype=np.int8)
next_actions = [(1, (1, 1, 1, 1)), (8, (0, 1, 1, 0)), (14, (1, 1, 1, 1)), (17, (1, 0, 1, 0)), (18, (1, 0, 1, 0))]
abs_action = (1,0,1,0)
[a for (a, aa) in next_actions if aa == abs_action]

[17, 18]

In [90]:
env = DiningPhilEnv(5)

In [122]:
state, info = env.reset()
print('vec:', state['vec'])
print('abs_vec:', state['abs_vec'])
print('=====')
#print(state['mask'])
next_actions = env._get_next_actions() 
#print(next_actions)
next_action = random.choice(next_actions)
next_abs_action = next_action[1]
print('next action:', next_action)
state, reward, done, info = env.abs_step(next_abs_action)
print(env._get_action_desc())
print('=====')
print('vec:', state['vec'])
print('abs_vec:', state['abs_vec'])

vec: (2, 2, 1, 3, 1, 0, 1, 0, 0, 0)
abs_vec: (0, 1, 1, 1)
=====
next action: (2, (0, 1, 1, 1))
philsopher 0 picks right chopstick
=====
vec: (3, 2, 1, 3, 1, 0, 0, 0, 0, 0)
abs_vec: (0, 1, 1, 1)


In [35]:
# check random initialization works fine (_generate_random_state)
for _ in range(30):
    state, info = env.reset()
    print(state['vec'])

(0, 2, 1, 2, 2, 0, 0, 1, 1, 0)
(1, 2, 1, 1, 1, 1, 1, 0, 1, 1)
(0, 3, 2, 2, 2, 0, 0, 0, 0, 0)
(2, 3, 2, 0, 0, 0, 0, 0, 0, 1)
(2, 1, 2, 2, 1, 1, 0, 0, 0, 1)
(3, 1, 3, 2, 1, 0, 0, 0, 0, 0)
(2, 2, 0, 0, 2, 1, 0, 0, 1, 0)
(2, 0, 2, 1, 2, 0, 0, 1, 0, 1)
(2, 3, 1, 2, 1, 0, 0, 0, 0, 1)
(3, 2, 2, 2, 1, 0, 0, 0, 0, 0)
(3, 2, 0, 3, 1, 0, 0, 0, 0, 0)
(0, 0, 2, 3, 2, 0, 1, 0, 0, 0)
(2, 2, 2, 0, 3, 0, 0, 0, 0, 0)
(2, 1, 1, 2, 1, 0, 1, 1, 0, 1)
(2, 0, 2, 2, 2, 0, 0, 0, 1, 0)
(2, 0, 2, 2, 2, 0, 0, 0, 1, 0)
(2, 3, 0, 2, 1, 0, 0, 0, 1, 0)
(1, 2, 2, 0, 2, 1, 1, 0, 0, 0)
(1, 2, 0, 2, 0, 1, 1, 0, 1, 0)
(2, 1, 3, 2, 2, 0, 0, 0, 0, 0)
(2, 2, 0, 0, 1, 0, 1, 0, 1, 1)
(0, 1, 2, 2, 2, 0, 1, 1, 0, 0)
(1, 0, 2, 2, 3, 0, 1, 0, 0, 0)
(1, 1, 1, 2, 0, 1, 1, 1, 1, 0)
(0, 3, 2, 1, 2, 1, 0, 0, 0, 0)
(2, 1, 0, 2, 1, 1, 0, 1, 1, 0)
(2, 2, 2, 2, 2, 0, 0, 0, 0, 0)
(2, 1, 2, 0, 3, 0, 0, 1, 0, 0)
(1, 2, 1, 2, 2, 1, 0, 1, 0, 0)
(2, 1, 2, 1, 3, 0, 0, 0, 1, 0)


* (2, 3, 2, 1, 0, 0, 0, 0, 0, 1)
* (True, True, True, True)
* [0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0]
* [None, None, None, None, None, None, None, (True, True, True, False), None, None, None, None, None, None, (True, False, True, True), None, (False, True, True, True), None, None, None]

In [91]:
# Let's create our Qtable of size (state_space, action_space) and initialized each values at 0 using np.zeros
def init_q_table(state_space, action_space):
  Qtable = np.zeros(state_space + action_space)
  return Qtable

In [5]:
Qtable = init_q_table(env.shape_state_space, env.shape_action_space)

In [92]:
absQtable = init_q_table((2,2,2,2), (2,2,2,2))

In [6]:
len(Qtable[(0,0,0,0,0,1,1,1,1,1)])

20

In [95]:
absQtable.shape

(2, 2, 2, 2, 2, 2, 2, 2)

In [53]:
q = np.array([1,2,3])
mask=[1,0,1]
mq = ma.masked_array(q, mask=mask)
print(mq)
print(np.argmax(mq))
print(0 in mask)

[-- 2 --]
1
True


In [63]:
mask=np.array([1,0,1])
new_mask=np.where(True, mask^1, mask)
print(new_mask)

[0 1 0]


In [7]:
def greedy_policy(Qtable, state):
    # Exploitation: take the action with the highest state, action value
    vec = state["vec"]
    mask = state["mask"]
    mask = np.where(True, mask^1, mask) # flip 0 & 1
    if 0 in mask:
        masked_actions = ma.masked_array(Qtable[vec][:], mask=mask)
        action = np.argmax(Qtable[vec][:]) # TODO error!!!!!!!!!
    else:
        action = -1 # deadlock
    return action

In [116]:
def epsilon_greedy_policy(Qtable, state, epsilon):
    # Randomly generate a number between 0 and 1
    random_num = random.uniform(0, 1)
    # if random_num > greater than epsilon --> exploitation
    if random_num > epsilon:
        # Take the action with the highest value given a state
        # np.argmax can be useful here
        action = greedy_policy(Qtable, state)
    # else --> exploration
    else:
        mask = state["mask"]
        if 1 in mask:
            action = env.action_space.sample(mask=mask) # TODO : masked sample
        else:
            action = -1
    return action

In [115]:
mask = env._get_abs_mask()
print(mask)
actions = np.transpose(np.nonzero(mask))
print(actions)
print(random.choice(actions))

[[[[0 0]
   [0 0]]

  [[0 0]
   [0 0]]]


 [[[0 0]
   [1 0]]

  [[0 0]
   [1 1]]]]
[[1 0 1 0]
 [1 1 1 0]
 [1 1 1 1]]
[1 1 1 0]


In [179]:
state, _ = env.reset()
vec = (1, 1, 0, 0)
mask = np.array([[[[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]]],


       [[[0, 0],
         [1, 0]],

        [[1, 0],
         [0, 0]]]], dtype=np.int8)
mask = np.where(True, mask^1, mask) # flip 0 & 1
print(mask)
print(absQtable[vec][:])
if 0 in mask:
    masked_Q = ma.masked_array(absQtable[vec][:], mask=mask)
    action = np.unravel_index(np.argmax(masked_Q), (2,2,2,2))
else:
    action = -1 # deadlock
print(action)

[[[[1 1]
   [1 1]]

  [[1 1]
   [1 1]]]


 [[[1 1]
   [0 1]]

  [[0 1]
   [1 1]]]]
[[[[0.       0.      ]
   [0.       0.      ]]

  [[0.9025   0.      ]
   [0.       0.      ]]]


 [[[0.       0.      ]
   [0.9025   0.      ]]

  [[0.857375 0.      ]
   [0.9025   0.      ]]]]
(1, 0, 1, 0)


In [154]:
testQ = np.array([[[1,2],[3,4]],[[5,6],[7,8]]])
mask = np.array([[0,0],[1,1]])
print('Q:', testQ[0][:])
print('Q shape:', testQ[0][:].shape)
masked_actions = ma.masked_array(testQ[0][:], mask=mask)
print(np.unravel_index(np.argmax(masked_actions), (2,2)))

Q: [[1 2]
 [3 4]]
Q shape: (2, 2)
(0, 1)


In [93]:
def abs_greedy_policy(Qtable, state):
    # Exploitation: take the action with the highest state, action value
    vec = state["abs_vec"]
    mask = state["abs_mask"]
    mask = np.where(True, mask^1, mask) # flip 0 & 1
    if 0 in mask:
        masked_Q = ma.masked_array(Qtable[vec][:], mask=mask)
        action = np.unravel_index(np.argmax(masked_Q), (2,2,2,2))
    else:
        action = -1 # deadlock
    return action

In [94]:
def abs_epsilon_greedy_policy(Qtable, state, epsilon):
    # Randomly generate a number between 0 and 1
    random_num = random.uniform(0, 1)
    # if random_num > greater than epsilon --> exploitation
    if random_num > epsilon:
        # Take the action with the highest value given a state
        # np.argmax can be useful here
        action = abs_greedy_policy(Qtable, state)
    # else --> exploration
    else:
        mask = state["abs_mask"]
        if 1 in mask:
            #action = env.abs_action_space.sample(mask=mask) # TODO : masked sample
            action = tuple(random.choice(np.transpose(np.nonzero(mask))))
        else:
            action = -1
    return action

In [95]:
# Training parameters
n_training_episodes = 100  # Total training episodes
learning_rate = 0.7  # Learning rate

# Evaluation parameters
n_eval_episodes = 100  # Total number of test episodes

# Environment parameters
#env_id = "FrozenLake-v1"  # Name of the environment
max_steps = 99  # Max steps per episode
gamma = 0.95  # Discounting rate
eval_seed = []  # The evaluation seed of the environment

# Exploration parameters
max_epsilon = 1.0  # Exploration probability at start
min_epsilon = 0.05  # Minimum exploration probability
decay_rate = 0.0005  # Exponential decay rate for exploration prob

In [54]:
a = []
a.append(1)
a

[1]

In [10]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
    for episode in tqdm(range(n_training_episodes)):
        # Reduce epsilon (because we need less and less exploration)
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        # Reset the environment
        state, info = env.reset()
        step = 0
        done = False

        # repeat
        for step in range(max_steps):
            # Choose the action At using epsilon greedy policy
            action = epsilon_greedy_policy(Qtable, state, epsilon)
            # assert action not -1

            # Take action At and observe Rt+1 and St+1
            # Take the action (a) and observe the outcome state(s') and reward (r)
            new_state, reward, done, info = env.step(action)
            
            vec = state["vec"]
            new_vec = new_state["vec"]

            # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
            Qtable[vec][action] = Qtable[vec][action] + learning_rate * (
                reward + gamma * np.max(Qtable[new_vec]) - Qtable[vec][action]
            )

            # If terminated or truncated finish the episode
            if done:
                break

            # Our next state is the new state
            state = new_state
    print('training done!')
    return Qtable

In [11]:
Qtable = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable)

  0%|          | 0/1000 [00:00<?, ?it/s]

training done!


In [12]:
print('Qtable density:', np.count_nonzero(Qtable), '/', Qtable.size)

Qtable density: 3250 / 655360


* N: 5, n_training_episodes: 1000 -> density: 3342 / 655360
* N: 5, n_training_episodes: 10000 -> density: 6224 / 655360

In [98]:
def abs_train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
    for episode in tqdm(range(n_training_episodes)):
        # Reduce epsilon (because we need less and less exploration)
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        # Reset the environment
        state, info = env.reset()
        step = 0
        done = False

        # repeat
        for step in range(max_steps):
            #print('===')
            #print(env._get_obs())
            # Choose the action At using epsilon greedy policy
            action = abs_epsilon_greedy_policy(Qtable, state, epsilon)
            # assert action not -1
            if action == -1:
                break
            
            #print(action)

            # Take action At and observe Rt+1 and St+1
            # Take the action (a) and observe the outcome state(s') and reward (r)
            new_state, reward, done, info = env.abs_step(action)
            
            #print(done)
            
            vec = state["abs_vec"]
            new_vec = new_state["abs_vec"]

            # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
            Qtable[vec][action] = Qtable[vec][action] + learning_rate * (
                reward + gamma * np.max(Qtable[new_vec]) - Qtable[vec][action]
            )

            # If terminated or truncated finish the episode
            if done:
                break

            # Our next state is the new state
            state = new_state
    print('training done!')
    return Qtable

In [99]:
absQtable = abs_train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, absQtable)

  0%|          | 0/100 [00:00<?, ?it/s]

(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 1)
(0, 1, 0, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 0, 1)
(1, 1, 0, 1)
(1, 1, 0, 1)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 0, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 0, 1)
(1, 1, 0, 1)
(1, 1, 1, 1)
(1, 1, 0, 1)
(1, 1, 0, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)

(1, 0, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 0, 1, 0)
(0, 1, 1, 0)
(0, 0, 1, 0)
(1, 1, 0, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 1)
(1, 1, 1, 1)
(1, 1, 0, 1)
(1, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 0, 0)
(0, 1, 0, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)

(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 0, 1, 0)
(1, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 1)
(0, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 0, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 0, 1, 0)
(1, 1, 1, 0)
(1, 1, 0, 1)
(0, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 0, 1)
(1, 1, 0, 0)
(0, 1, 0, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 0, 0)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 1, 1, 0)

(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 0)
(0, 1, 1, 0)
(0, 0, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 0, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 0, 1)
(1, 0, 0, 0)
(1, 1, 0, 0)
(1, 0, 1, 0)
(1, 0, 0, 1)
(1, 0, 0, 0)
(1, 1, 0, 0)
(1, 0, 1, 0)
(1, 0, 0, 1)
(1, 1, 0, 1)
(1, 0, 1, 1)
(1, 1, 1, 1)
(1, 1, 0, 1)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 0, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 0, 0)
(1, 1, 0, 0)
(1, 1, 1, 0)
(1, 1, 0, 1)
(0, 1, 0, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)

...

(0, 1, 1, 0)
(0, 0, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 0, 0)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 1)
(0, 1, 0, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 0, 1)
(0, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)

(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 0, 1)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)

(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 0)
(0, 1, 1, 0)
(0, 0, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 0, 1)
(1, 1, 0, 0)
(0, 1, 0, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 0, 1)
(1, 1, 0, 1)
(0, 1, 0, 1)
(1, 1, 0, 0)
(0, 1, 0, 0)
(0, 1, 1, 0)
(0, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 0, 1)
(0, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)

(1, 0, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 0, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 0, 1)
(1, 1, 0, 1)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 0, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 0, 1)
(1, 1, 0, 1)
(0, 1, 0, 1)

(0, 1, 0, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 1)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(0, 1, 1, 1)
(0, 1, 0, 1)
(0, 1, 1, 1)
(1, 1, 1, 1)
(0, 1, 1, 1)
(0, 1, 0, 1)
(0, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 0, 0, 1)
(1, 1, 0, 1)
(1, 1, 0, 1)
(1, 1, 0, 1)
(0, 1, 0, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 0)
(1, 0, 1, 1)
(1, 0, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 1, 0)
(1, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 0, 0)
(1, 1, 0, 0)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 1, 1, 1)
(1, 0, 1, 1)
(1, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)

In [101]:
print('Qtable density:', np.count_nonzero(absQtable), '/', absQtable.size)

Qtable density: 44 / 256


In [13]:
# e-greedy Simulator

def simulate(Qtable, max_steps=100, epsilon=0.01):
    state, info = env.reset()
    # print(state['vec'])
    # print(info['action_desc'])
    # print('=================')

    for step in range(max_steps):
        # Choose the action At using epsilon greedy policy
        action = epsilon_greedy_policy(Qtable, state, epsilon)

        # Take action At and observe Rt+1 and St+1
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)
        # print(new_state['vec'])
        # print(info['action_desc'])
        # print('=================')

        # If terminated or truncated finish the episode
        if done:
            break

        # Our next state is the new state
        state = new_state
        
    return step + 1

In [16]:
res_trained = []
res_random = []
for _ in range(100):
    num_steps_trained = simulate(Qtable, epsilon=0)
    num_steps_random = simulate(Qtable, epsilon=1)
    res_trained.append(num_steps_trained)
    res_random.append(num_steps_random)
print('trained:')
print(res_trained)
print('random:')
print(res_random)

trained:
[7, 7, 7, 10, 7, 9, 3, 9, 6, 7, 10, 8, 8, 7, 2, 10, 9, 3, 9, 8, 5, 2, 100, 9, 3, 5, 8, 7, 6, 100, 8, 8, 5, 11, 8, 8, 5, 10, 10, 2, 4, 6, 9, 6, 10, 7, 7, 10, 8, 10, 2, 7, 7, 5, 12, 7, 6, 1, 8, 8, 8, 5, 8, 3, 8, 10, 5, 8, 11, 5, 4, 100, 9, 8, 8, 11, 4, 7, 100, 7, 7, 4, 9, 2, 9, 8, 9, 8, 8, 6, 4, 5, 8, 1, 5, 5, 6, 10, 100, 4]
random:
[17, 100, 30, 100, 5, 63, 23, 69, 100, 19, 13, 100, 61, 36, 92, 30, 5, 100, 16, 42, 100, 40, 100, 12, 31, 30, 100, 26, 100, 15, 33, 100, 97, 100, 3, 90, 76, 23, 100, 100, 50, 49, 20, 31, 24, 58, 94, 100, 100, 29, 100, 54, 100, 79, 51, 2, 100, 66, 31, 62, 9, 77, 100, 39, 18, 15, 100, 55, 100, 33, 99, 10, 75, 100, 84, 85, 100, 69, 49, 13, 100, 6, 56, 100, 16, 100, 100, 73, 27, 69, 12, 100, 100, 51, 10, 38, 63, 38, 2, 57]


In [102]:
# e-greedy Simulator
def abs_simulate(Qtable, max_steps=100, epsilon=0.01):
    state, info = env.reset()
    # print(state['vec'])
    # print(info['action_desc'])
    # print('=================')

    for step in range(max_steps):
        #print('===')
        #print(env._get_obs())
        # Choose the action At using epsilon greedy policy
        action = abs_epsilon_greedy_policy(Qtable, state, epsilon)
        if action == -1:
            break

        # Take action At and observe Rt+1 and St+1
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.abs_step(action)
        # print(new_state['vec'])
        # print(info['action_desc'])
        # print('=================')
        #print(done)

        # If terminated or truncated finish the episode
        if done:
            break

        # Our next state is the new state
        state = new_state
        
    return step + 1

In [103]:
env5= DiningPhilEnv(5)
env6= DiningPhilEnv(6)
env7= DiningPhilEnv(7)
env8= DiningPhilEnv(8)
env9= DiningPhilEnv(9)
env10= DiningPhilEnv(10)

In [104]:
env = env5
max_steps = 500
res_trained = []
res_random = []
for _ in range(100):
    num_steps_trained = abs_simulate(absQtable, max_steps=max_steps, epsilon=0)
    num_steps_random = abs_simulate(absQtable, max_steps=max_steps, epsilon=1)
    res_trained.append(num_steps_trained)
    res_random.append(num_steps_random)
print('trained:')
print(res_trained)
print('avg:')
print(np.average(res_trained))
print('=====')
print('random:')
print(res_random)
print('avg:')
print(np.average(res_random))

trained:
[5, 24, 21, 5, 7, 21, 16, 23, 14, 44, 10, 91, 72, 8, 23, 8, 14, 27, 3, 103, 9, 22, 31, 30, 15, 6, 1, 20, 27, 1, 27, 15, 11, 13, 11, 32, 92, 6, 10, 27, 20, 4, 43, 4, 16, 4, 5, 6, 19, 1, 6, 12, 6, 17, 4, 51, 22, 53, 19, 15, 7, 7, 9, 36, 18, 7, 18, 26, 10, 31, 2, 6, 27, 4, 8, 3, 16, 22, 18, 12, 8, 10, 8, 13, 7, 15, 2, 4, 12, 5, 23, 31, 30, 30, 11, 8, 9, 6, 12, 34]
avg:
18.37
=====
random:
[37, 23, 313, 94, 500, 260, 39, 368, 115, 98, 78, 103, 182, 10, 3, 143, 53, 131, 124, 191, 312, 27, 74, 59, 26, 196, 320, 13, 211, 78, 204, 55, 140, 194, 82, 107, 338, 13, 147, 208, 36, 473, 403, 252, 44, 138, 78, 226, 16, 146, 133, 7, 500, 23, 500, 131, 41, 411, 99, 134, 142, 283, 58, 8, 93, 154, 38, 77, 7, 188, 328, 36, 53, 93, 114, 76, 46, 14, 39, 3, 78, 171, 409, 311, 193, 114, 60, 30, 11, 53, 148, 101, 10, 112, 148, 298, 54, 75, 314, 177]
avg:
141.77


In [5]:
import heapq
q = []
heapq.heappush(q, (-2,'v3'))
heapq.heappush(q, (-4,'v1'))
heapq.heappush(q, (-3,'v2'))
print(heapq.heappop(q))
print(heapq.heappop(q))
print(heapq.heappop(q))

(-4, 'v1')
(-3, 'v2')
(-2, 'v3')


In [9]:
a = []
not a == []

False

In [7]:
maude.init()
maude.load('./example.maude')
m = maude.getCurrentModule()
print('Using', m, 'module')

Using EXAMPLE module


In [11]:
for i in tqdm(range(10)):
    print('hi')

  0%|          | 0/10 [00:00<?, ?it/s]

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


In [8]:
term = m.parseTerm('f(a, a)')

In [39]:
state_space = ((4,)* env.N) + ((2,)* env.N)
action_space = (4 * env.N,)
print(env.shape_state_space + env.shape_action_space)

(4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 20)


In [14]:
env = MaudeEnv(term)

/home/byhoson/.local/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [15]:
env.observation_space.sample()

array([96.86537], dtype=float32)

In [56]:
mask = np.ones(20, dtype=np.int8)
#len(mask)
for i in range(30):
    print(env.action_space.sample(mask=mask))

11
1
10
4
19
6
12
12
15
3
16
3
19
17
0
0
7
1
1
8
17
18
1
11
7
15
1
10
5
15


In [16]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

IndexError: list index out of range

In [11]:
nbrs = [t for t,_,_,_ in term.search(1, m.parseTerm('X:Symbol'), depth = 1)]

In [12]:
nbrs

[f(b, a), f(c, a), f(a, b), f(a, c)]

In [ ]:
class MaudeState(Vertex):
    """Create a Maude term
    
    Args:

    """
    
    def __init__(
        self,
        module,
        pattern,
        history: List[maude.Term],
        symbols # list of strings e.g. ['f', 'g'] fixed a priori
    ) -> None:
        
        super().__init__()
        
        self.m = module # Maude module
        self.pattern = pattern # goal pattern
        self.history = history # current term
        self.symbols = symbols
        self.ANY_PAT = m.parseTerm('X:Symbol')
        self.zeros = dict.fromkeys(symbols, np.zeros(1))
        
    @property
    def observation_space(self) -> gym.spaces.Dict:
        """Returns the graph env's observation space.

        Returns:
            Dict observation space.
        """
        """
        num_symbols = len(self.symbols)
        return gym.spaces.Dict(
            {
                "term_obs": gym.spaces.Box(
                    low=np.zeros(num_symbols), high=np.ones(num_symbols), dtype=int
                )
            }
        )
        """
        return gym.spaces.Dict(
            { sym : gym.spaces.Box(low=np.array([0]), high=np.array([999])) for sym in self.zeros }
        )

    @property
    def reward(self) -> float:
        """Returns the graph env reward.

        Returns:
            Negative distance between last two nodes in the tour.
        """

        cur_term = self.history[-1]
        match = cur_term.match(self.pattern)
        try:
            next(match)
            rew = 999999999
            
        except:
            rew = -1

        return rew

    @property
    def info(self) -> Dict:
        return {}

    def _get_children(self) -> Sequence["MaudeState"]:
        """Yields a sequence of TSPState instances associated with the next
        accessible nodes.

        Yields:
            New instance of the TSPState with the next node added to
            the tour.
        """

        cur_term = self.history[-1]
        seq_terms = cur_term.search(1, self.ANY_PAT, depth = 1)

            
        nbrs = [t for t,_,_,_ in seq_terms if t not in self.history]
        print('cur term:', cur_term)
        print("nbrs:", nbrs)
        
        # Loop over the neighbors and update paths.
        for next_term in nbrs:
            history = self.history.copy()
            history.append(next_term)
            #print('history:', history, 'next_term:', next_term)
            yield self.__class__(module=self.m, pattern=self.pattern, history=history, symbols = self.symbols)

    def _make_observation(self) -> Dict[str, np.ndarray]:
        """Return an observation.  The dict returned here needs to match
        both the self.observation_space in this class, as well as the input
        layer in tsp_model.TSPModel

        Returns:
            Observation dict.  We define the node_obs to be the degree of the
            current node.  This is a placeholder for a more meaningful feature!
        """

        # get bag of words of the current term
        cur_term = self.history[-1]
        return self.term2dict(cur_term)
    
    def term2dict(self, term):
        def getSymbols(term):
            ret = [getSymbols(arg) for arg in term.arguments()]
            ret = [x for xs in ret for x in xs]
            ret += [str(term.symbol())]
            return ret
        counter = Counter(getSymbols(term))
        counter = { k : np.array([counter[k]]) for k in counter }
        #return { k : self.zeros.get(k, 0) + counter.get(k, 0) for k in set(counter) | set(self.zeros)}
        return { k : self.zeros.get(k, 0) + counter.get(k, 0) for k in set(self.zeros)}
    
    def __deepcopy__(self, memo):
        return self.__class__(module=self.m, pattern=self.pattern, history=self.history[:], symbols = self.symbols)
    
    def __getstate__(self):
        return {'pattern': 'hello'}
    
    def __setstate__(self, state):
        self.pattern = state['pattern']
